In [ ]:
import subprocess
import sys

def detup_sweep(gpu_num, config_name, sweep_id=None):
    if sweep_id is None:
        result = subprocess.run(['wandb', 'sweep', f'configs/{config_name}.yaml'], capture_output=True, text=True)
        sweep_id = str(result).split('/')[-1].split('\\')[0]
    script = f"""#!/bin/bash
    
    source ~/.bash_profile 2>/dev/null || source ~/.bashrc
    eval "$(conda shell.bash hook)"
    conda activate evodiff
    wandb agent --name {config_name} alanamin/d3pm/{sweep_id}"""
    script_path = f'scripts/{config_name}_{sweep_id}.sh'
    with open(script_path, 'w+') as f:
        f.write(script)
    subprocess.run(['chmod', '+x', script_path])
    print("Sweep id:", sweep_id, "\nCommand:")
    print(f"cd ~/d3pm; nohup bash -c 'CUDA_VISIBLE_DEVICES={gpu_num} ~/d3pm/{script_path}' > ~/d3pm/outputs/{config_name}_{sweep_id}.log 2>&1 &")

In [ ]:
gpu_num = 1
config_name = 'sweep_gamma_cifar_params_agent_2'
sweep_id = "x38oy66z"
detup_sweep(gpu_num, config_name, sweep_id)

## slurm

In [ ]:
import numpy as np
import os
import subprocess
import wandb

def create_slurm_job(config_name, sweep_id=None, a100=False):
    # Create a new wandb sweep if sweep_id is not provided
    if sweep_id is None:
        result = subprocess.run(['wandb', 'sweep', '--name', config_name, f'configs/{config_name}.yaml'], capture_output=True, text=True)
        print(result)
        sweep_id = str(result).split('/')[-1].split('\\')[0]

    rand = np.random.randint(1000000)
    # Create a SLURM job script
    slurm_script = f"""#!/bin/bash
#SBATCH --job-name=sweeps{rand}
#SBATCH --output=slurm_out/slurm_{rand}_%j.out
#SBATCH --error=slurm_out/slurm_{rand}_%j.err
#SBATCH --time=24:00:00
#SBATCH --ntasks=1
#SBATCH --mem=100GB
#SBATCH --cpus-per-task=16
#SBATCH --gres=gpu:{'a100:' if a100 else ''}1


# Activate your virtual environment if needed
cd $SCRATCH/d3pm
source ~/.bashrc
conda activate evodiff

# Run the wandb agent
wandb agent alanamin/d3pm/{sweep_id}
"""

    # Write the SLURM script to a file
    script_filename = f"slurm_scripts/slurm_job_{sweep_id}_{rand}.sh"
    with open(script_filename, 'w') as f:
        f.write(slurm_script)

    # Submit the SLURM job
    # print('sbatch ' + script_filename)
    subprocess.run(['sbatch', script_filename])

    return sweep_id


In [ ]:
config_name = 'sweep_process_lm1b'
sweep_id = None
create_slurm_job(config_name, sweep_id, a100=True)

In [ ]:
config_name = 'sweep_gamma_cifar_gaussian'
sweep_id = None
create_slurm_job(config_name, sweep_id)

In [ ]:
config_name = 'sweep_gamma_cifar_big'
sweep_id = None
create_slurm_job(config_name, sweep_id)

In [ ]:
for i in range(17):
    create_slurm_job(None, 'f944m4om')

In [ ]:
config_name = 'baselines/sweep_gamma_cifar_temb_trial'
sweep_id = None
create_slurm_job(config_name, sweep_id)

## Make masking and sedd

In [ ]:
import yaml

def make_baseline_configs(config_name):
    # Load the existing configuration
    with open(f'configs/{config_name}.yaml', 'r') as file:
        config = yaml.safe_load(file)
    
    if 'model.gamma' in config['parameters']:
        del config['parameters']['model.gamma']
    
    config['parameters']['architecture.nn_params.time_embed_dim'] = {'value': 512}
    config['parameters']['model.model'] = {'values': ['MaskingDiffusion', 'SEDD']}
    
    # Save the modified configuration
    with open(f'configs/baselines/{config_name}.yaml', 'w') as file:
        yaml.dump(config, file, default_flow_style=False)
    return f'baselines/{config_name}'

In [ ]:
config_name = 'sweep_gamma_cifar_big'
config_name = make_baseline_configs(config_name)
create_slurm_job(config_name, None)

In [ ]:
config_name = 'sweep_gamma_cifar_gaussian'
config_name = make_baseline_configs(config_name)
create_slurm_job(config_name, None)